In [ ]:
import pandas as pd
import re
import datetime
import json
from datetime import datetime
import time
import warnings
warnings.filterwarnings("ignore")

#importorting libraries for NLP
from nltk.corpus import stopwords
stop = stopwords.words('english')
from textblob import TextBlob
import string
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
from statistics import mode


#importing librraies for data modelling 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics 
from mlxtend.evaluate import bias_variance_decomp
import xgboost as xg


profile=pd.read_csv('instagram_profiles.csv')
profile_post=pd.read_csv('instagram_posts.csv')

profile_post['hashtags'].fillna(0,inplace=True)
meta_list=list(profile_post['metadata'])
caption_is_question_list=[]
time_list=[]
caption_list=[]
text_caption_list=[]
hashtag_list=[]
day_of_week_list=[]
hour_list=[]
post_type_list=[]

#creating general features

def timestamp(i,j):
    
    global time_list
    global caption_list
    global text_caption_list
    global hashtag_list
    global day_of_week_list
    global hour_list
    global post_type_lists
    
    
    res = json.loads(i)
    time_value=res['timestamp']
    post_type=res["hyp_type"]
    day_of_week=pd.Timestamp(time_value).dayofweek
    hour_of_day=pd.Timestamp(time_value).hour
    day_of_week_list.append(day_of_week)
    hour_list.append(hour_of_day)
    post_type_list.append(post_type)
    
    
    if 'caption' in list(res.keys()):
        caption_value=res['caption']
        text=caption_value.split('\n')[0]
        text_caption_list.append(text)
        if '?'in text:
            caption_is_question='Yes'
        else:
             caption_is_question='No'
        caption_list.append(caption_value)
        time_list.append(time_value)
        caption_is_question_list.append(caption_is_question)
        
    else:
        
        time_list.append(time_value)
        caption_list.append('No Caption')
        text_caption_list.append('No Caption')
        caption_is_question_list.append('No')
        
    if j!=0:
        no_of_hashtags=len(j.strip('{}').split(','))
        hashtag_list.append(no_of_hashtags)
    else:
        no_of_hashtags=0
        hashtag_list.append(no_of_hashtags)
        
    

profile_post.apply(lambda x: timestamp(x['metadata'],x['hashtags']),axis=1)
profile_post['timestamp']=time_list
profile_post['Caption']=caption_list
profile_post['Text_Caption']=text_caption_list
profile_post['No_of_Hashtags']=hashtag_list
profile_post['day_of_week']=day_of_week_list
profile_post['Hour']=hour_list
profile_post['post_type']=post_type_list
profile_post['caption_is_question']=caption_is_question_list

final_creater_dataframe=profile_post.merge(profile,on='ig_id')
final_creater_dataframe = final_creater_dataframe.sort_values(by=["ig_id","timestamp"])

In [213]:
creator=list(set(final_creater_dataframe['username']))

#check the brightness of an image /video
import numpy as np
def isbright(image, dim=10, thresh=0.6):
    # Resize image to 10x10
    image = cv2.resize(image, (dim, dim))
    # Convert color space to LAB format and extract L channel
    L, A, B = cv2.split(cv2.cvtColor(image, cv2.COLOR_BGR2LAB))
    # Normalize L channel by dividing all pixel values with maximum pixel value
    L = L/np.max(L)
    #print(np.mean(L))
    # Return True if mean is greater than thresh else False
    return np.mean(L) >= thresh



In [ ]:
import glob
import cv2

#using haarcascade from computer visison to detectect smile , face and brightness of a image 

face_cascade=cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml") #Note the change
smile_detector = cv2.CascadeClassifier(cv2.data.haarcascades +"haarcascade_smile.xml")

#image_feature=pd.DataFrame()
for x in creator[485:]:
        print(x)
        data=final_creater_dataframe[final_creater_dataframe['username']==x]
        all_files_images = glob.glob('D:/DS_Assignment_Data/{}/*jpg'.format(x))
        image_name=all_files[1].split('\\')[-1]
        
        creator_face_detection=[]
        all_file_videos=glob.glob('D:/DS_Assignment_Data/{}/*mp4'.format(x))
        for i in all_files:
            
        
            img = cv2.imread(i)
            gray_img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            faces=face_cascade.detectMultiScale(gray_img, scaleFactor=1.05,minNeighbors=5)
            if len(faces)>0:
                detect=1
            else :
                detect=0
                
            creator_face_detection.append(detect)
            mode_value_detection=mode(creator_face_detection)
            
        creater_brightness=[]
        for j in all_file_videos:
            cap = cv2.VideoCapture(j) 
            for i in range(0,1): 
                # Read the frame  
                _, img = cap.read()  

                # Convert to grayscale  
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  

                # Detect the faces  
                faces = face_cascade.detectMultiScale(gray, 1.1, 4)  
            
            s=isbright(img, dim=10, thresh=0.7)
            creater_brightness.append(s)
            mode_value_brightness=mode(creater_brightness)
                                       
        new_row={'username':x,'brightness':mode_value_brightness,'face_detection':mode_value_detection}
        image_feature=image_feature.append(new_row,ignore_index=True)            

In [ ]:
image_feature.to_csv('Image_feature.csv')